In [11]:
# %%
import ee
import geemap
import geopandas as gpd



# Authenticate once (browser popup). After this, ee.Initialize() works.
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

print("✅ Earth Engine initialized")


✅ Earth Engine initialized


In [ ]:
# Authenticate once (browser popup). After this, ee.Initialize() works.
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

print("✅ Earth Engine initialized")

In [12]:
# %%


AOI_GEOJSON = r"C:\GIS\my_water_projects\openwater-shrinking-lake-monitor\data\external\aoi.geojson"

gdf = gpd.read_file(AOI_GEOJSON)
print("CRS:", gdf.crs)




CRS: EPSG:4326


In [ ]:
minx, miny, maxx, maxy = gdf.total_bounds  # uses your original gdf
aoi_bbox = ee.Geometry.Rectangle([minx, miny, maxx, maxy])

print("✅ Using bbox rectangle aoi_bbox")
print([minx, miny, maxx, maxy])

✅ Using bbox rectangle AOI
[np.float64(-107.21444572778415), np.float64(33.14571033916773), np.float64(-107.14274375499289), np.float64(33.31855197156376)]


In [15]:
# %%
def mask_s2_sr_clouds(img: ee.Image) -> ee.Image:
    """
    Basic cloud mask for Sentinel-2 Surface Reflectance (S2_SR_HARMONIZED) using QA60 bits.
    Bit 10 = clouds, Bit 11 = cirrus.

    Returns a reflectance-scaled image in 0–1 (divide by 10000).
    """
    qa = img.select("QA60")
    cloud_bit = 1 << 10
    cirrus_bit = 1 << 11

    mask = qa.bitwiseAnd(cloud_bit).eq(0).And(qa.bitwiseAnd(cirrus_bit).eq(0))
    return img.updateMask(mask).divide(10000)


def rgb_monthly_composite(year: int, month: int, aoi: ee.Geometry) -> ee.Image:
    """
    Build a median RGB composite for a given (year, month) over the AOI.
    Uses lenient CLOUDY_PIXEL_PERCENTAGE filtering, then a QA60 mask.
    """
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, "month")

    col = (
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterBounds(aoi)
        .filterDate(start, end)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 80))  # broad filter
        .map(mask_s2_sr_clouds)
    )

    # Median composite reduces residual cloud noise and fills coverage gaps across the month
    comp = col.median().clip(aoi)

    return comp


In [17]:
# %%
# Quick visual sanity check in an interactive map
Map = geemap.Map()
Map.centerObject(aoi_bbox, 11)

img_2019 = rgb_monthly_composite(2019, 9, aoi_bbox)
img_2025 = rgb_monthly_composite(2025, 9, aoi_bbox)

vis = {"min": 0.02, "max": 0.30, "bands": ["B4", "B3", "B2"]}

Map.addLayer(img_2019, vis, "2019-09 RGB (median)")
Map.addLayer(img_2025, vis, "2025-09 RGB (median)")
Map.addLayer(aoi_bbox, {}, "AOI Bounding Box")
Map


Map(center=[33.23210782905315, -107.17859474138854], controls=(WidgetControl(options=['position', 'transparent…

In [18]:
# %%
# Export to Google Drive as GeoTIFFs
# These become tasks you can monitor in the notebook output or in the EE Tasks UI.

def export_rgb_to_drive(img: ee.Image, aoi: ee.Geometry, name: str, folder: str = "openwater_rgb") -> None:
    geemap.ee_export_image_to_drive(
        image=img.select(["B4", "B3", "B2"]),
        description=name,
        folder=folder,
        fileNamePrefix=name,
        region=aoi,
        scale=10,
        maxPixels=1e13,
    )

export_rgb_to_drive(img_2019, aoi_bbox, "s2_rgb_2019_09")
export_rgb_to_drive(img_2025, aoi_bbox, "s2_rgb_2025_09")

print("✅ Drive export tasks created.")
print("   Google Drive → My Drive → openwater_rgb/")


✅ Drive export tasks created.
   Google Drive → My Drive → openwater_rgb/
